In [126]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import (train_test_split, KFold, cross_val_score, 
GridSearchCV, StratifiedKFold)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer


# Predicciones

Este notebook es el que sera entregado a la persona tecnica del banco que se encaragara de revisarlo de manera minuciosa y a nivel muy tecnico. A continuacion se presenta un bosquejo de las secciones que debe contener.  
Puedes agregar tantas celdas como requieras, pero manten la estructura general del proyecto.

Deberas utilizar `autogluon` para realizar el proyecto, sin embargo debes de ser mas cuidadosx en la seleccion de variables que lo que fuimos en el automl, puedes usar las variables que utilizamos en las predicciones de `sklearn`.

Pon especial atencion en tus sets, recuerda que debes entrenar el modelo, ver la calibracion, sugerir tasas de interes y simular como se va a comportar en la realidad. Por ende, debes de ser extremadamente cuidadosx en como los separas. Una mala separacion de estos sets arruinaria todo el proyecto, pues seria inutil.

## ETL

En esta seccion deberas cargas los datos desde el github de Lending Club

In [118]:
loans = pd.read_parquet('loans_sampe_data.parquet')
metadata = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')
metadata.columns = ['feature', 'description']
loans['loan_status'].value_counts(dropna=False)

loan_status
Fully Paid            69982
Charged Off           16156
Current               12726
Late (31-120 days)      730
In Grace Period         264
Late (16-30 days)       139
Default                   3
Name: count, dtype: int64

In [119]:
df = loans.loc[loans['loan_status'].isin(['Fully Paid', 
                                                   'Charged Off'])]
df['loan_status'].value_counts(normalize=True, dropna=False)

loan_status
Fully Paid     0.812441
Charged Off    0.187559
Name: proportion, dtype: float64

In [120]:
df['charged_off'] = (df['loan_status'] == 'Charged Off').apply(np.uint8)
df.drop('loan_status', axis=1, inplace=True)
df['charged_off'].unique()

/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/4139191855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['charged_off'] = (df['loan_status'] == 'Charged Off').apply(np.uint8)
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/4139191855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('loan_status', axis=1, inplace=True)


array([0, 1], dtype=uint8)

## Feature Selection

In [121]:
elegidas = ['charged_off','funded_amnt','addr_state', 'annual_inc',
            'application_type', 'dti', 'earliest_cr_line', 'emp_length', 
            'emp_title', 'fico_range_high', 'fico_range_low', 'grade',
            'home_ownership', 'initial_list_status', 'installment', 
            'loan_amnt', 'loan_status', 'mort_acc', 
            'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 
            'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 
            'total_acc', 'verification_status', 'last_pymnt_amnt','num_actv_rev_tl', 'mo_sin_rcnt_rev_tl_op',
            'mo_sin_old_rev_tl_op',"bc_util","bc_open_to_buy","avg_cur_bal",
            "acc_open_past_24mths" ]
len(elegidas)

37

In [122]:
drop_no_intuitivas = [col for col in df.columns if col not in elegidas]
df.drop(labels=drop_no_intuitivas , axis=1, inplace=True)
df

/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/2027118353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(labels=drop_no_intuitivas , axis=1, inplace=True)


,loan_amnt,funded_amnt,term,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mort_acc,num_actv_rev_tl,pub_rec_bankruptcies,charged_off
0,15000.0,15000.0,60 months,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,...,5.0,29828.0,9525.0,4.7,244.0,1.0,0.0,4.0,0.0,0
1,10400.0,10400.0,36 months,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,...,7.0,9536.0,7599.0,41.5,290.0,1.0,1.0,9.0,0.0,1
2,21425.0,21425.0,60 months,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800.0,...,4.0,4232.0,324.0,97.8,136.0,7.0,0.0,4.0,0.0,0
4,7650.0,7650.0,36 months,260.20,C,C3,Technical Specialist,< 1 year,RENT,50000.0,...,6.0,5857.0,332.0,93.2,148.0,8.0,0.0,4.0,0.0,1
5,9600.0,9600.0,36 months,326.53,C,C3,Admin Specialist,10+ years,RENT,69000.0,...,8.0,3214.0,6494.0,69.2,265.0,23.0,0.0,7.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,15000.0,15000.0,36 months,485.07,B,B2,Senior Clinical Business Solutions Analy,10+ years,MORTGAGE,84136.0,...,5.0,14311.0,28091.0,31.3,356.0,3.0,5.0,5.0,0.0,0
99995,8400.0,8400.0,36 months,267.79,B,B1,Software Engineer,2 years,MORTGAGE,91000.0,...,2.0,25364.0,2505.0,77.2,287.0,5.0,1.0,2.0,0.0,0
99996,10000.0,10000.0,36 months,336.90,C,C1,Assistant Professor of English,3 years,RENT,53133.0,...,3.0,4219.0,13974.0,42.5,158.0,5.0,0.0,5.0,0.0,0
99998,8475.0,8475.0,36 months,336.92,F,F4,Records Examiner Analyst,10+ years,RENT,52000.0,...,8.0,3125.0,324.0,88.0,164.0,7.0,0.0,3.0,1.0,1


## Feature selection and feature engineering

In [123]:
def _check_type(value):
    ''' 
    Infiere si la variable es categorica o numerica
    apartir del tipo
    '''
    if value==object:
        return 'categorica'
    elif value==bool:
        return 'categorica' 
    elif value=='int64':
        return 'numerica'
    elif value=='float64':
        return 'numerica'
    
feature_type = pd.DataFrame(df.dtypes)
feature_type[0] = feature_type[0].apply(lambda x: _check_type(x))
feature_type.reset_index(inplace=True)
feature_type.rename(columns={'index':'feature',0:'tipo'},inplace=True)

features_cat = feature_type[feature_type['tipo']=='categorica']['feature']
features_cat = list(features_cat)
features_cat



['term',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'title',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'application_type']

In [124]:
cat_not_inclue = ['emp_title', 'application_type',
              'title', 'addr_state', 'pub_rec', 'revol_bal',
              'total_acc', 'pub_rec_bankruptcies']
features_cat = [f for f in features_cat if f not in cat_not_inclue]
df.drop(cat_not_inclue, axis=1, inplace=True)
df[features_cat] = df[features_cat].fillna('nan-missing')
df


/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/3633460319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(cat_not_inclue, axis=1, inplace=True)
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/3633460319.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features_cat] = df[features_cat].fillna('nan-missing')


,loan_amnt,funded_amnt,term,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,last_pymnt_amnt,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mort_acc,num_actv_rev_tl,charged_off
0,15000.0,15000.0,60 months,336.64,C,C1,10+ years,RENT,78000.0,Source Verified,...,12017.81,5.0,29828.0,9525.0,4.7,244.0,1.0,0.0,4.0,0
1,10400.0,10400.0,36 months,321.08,A,A3,8 years,MORTGAGE,58000.0,Not Verified,...,321.08,7.0,9536.0,7599.0,41.5,290.0,1.0,1.0,9.0,1
2,21425.0,21425.0,60 months,516.36,D,D1,6 years,RENT,63800.0,Source Verified,...,17813.19,4.0,4232.0,324.0,97.8,136.0,7.0,0.0,4.0,0
4,7650.0,7650.0,36 months,260.20,C,C3,< 1 year,RENT,50000.0,Source Verified,...,17.70,6.0,5857.0,332.0,93.2,148.0,8.0,0.0,4.0,1
5,9600.0,9600.0,36 months,326.53,C,C3,10+ years,RENT,69000.0,Source Verified,...,9338.58,8.0,3214.0,6494.0,69.2,265.0,23.0,0.0,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,15000.0,15000.0,36 months,485.07,B,B2,10+ years,MORTGAGE,84136.0,Not Verified,...,10584.91,5.0,14311.0,28091.0,31.3,356.0,3.0,5.0,5.0,0
99995,8400.0,8400.0,36 months,267.79,B,B1,2 years,MORTGAGE,91000.0,Source Verified,...,267.50,2.0,25364.0,2505.0,77.2,287.0,5.0,1.0,2.0,0
99996,10000.0,10000.0,36 months,336.90,C,C1,3 years,RENT,53133.0,Not Verified,...,5594.78,3.0,4219.0,13974.0,42.5,158.0,5.0,0.0,5.0,0
99998,8475.0,8475.0,36 months,336.92,F,F4,10+ years,RENT,52000.0,Verified,...,336.92,8.0,3125.0,324.0,88.0,164.0,7.0,0.0,3.0,1


In [125]:
df['term'] = df['term'].apply(lambda s: np.int8(s.split()[0]))

df.drop(['emp_length'], axis=1, inplace=True)

label_f = LabelEncoder()
lf = df['sub_grade'].apply(str).unique()
lf[-1] = 'missing-nan'
label_f.fit(lf)
df['sub_grade'] = label_f.fit_transform(df['sub_grade'])

df.drop(['grade'], axis=1, inplace=True)

df = df[df['home_ownership']!='ANY']



/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/472981632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['term'] = df['term'].apply(lambda s: np.int8(s.split()[0]))
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/472981632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['emp_length'], axis=1, inplace=True)
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/472981632.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [128]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
a = enc.fit_transform(X=df[['home_ownership']])
a = pd.DataFrame(a)
a.columns = enc.get_feature_names_out()
df[a.columns] = a.values
df.drop(columns=['home_ownership_OWN', 'home_ownership'], inplace=True)


/Users/elcachorrohumano/anaconda3/envs/automl_env/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/2751773607.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[a.columns] = a.values
/var/folders/2x/gfyvyqkn6m14010xg_kz1xr80000gn/T/ipykernel_27436/2751773607.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
a = enc.fit_transform(X=df[['verification_status']])
a = pd.DataFrame(a)
a.columns = enc.get_feature_names_out()
df[a.columns] = a.values
df.drop(columns=['verification_status'], inplace=True)

## Entrenamiento

Codigo de entrenamiento de modelos con `autogluon`. Toma en cuenta el imbalance de clases y las peticiones que tiene el banco.

## Seleccion de Modelos

En esta seccion debes elegir un modelo apra cada caso, y explicar cuidadosamente por que elegiste cada modelo, ademas de indicar que metricas utilizaste para elegir.

Pon especial atencio a las metricas que elegiste, y justifica por que estas metricas son importantes/buenas para el problema. Recuerda que este notebook sera revisado por una persona tecnica.

Incluye graficas, tablas o representacions visuales para explicar por que elgiste cada modelo y las metricas que utilizaste para elegir el modelo.

Salva tus modelos, pues los utilizaras en el siguiente notebook. Para salvar el modelo utiliza la funcion https://auto.gluon.ai/stable/api/autogluon.tabular.TabularPredictor.clone_for_deployment.html 

### Modelo Interpretable

#### Calibracion del modelo

En esta seccion revisa la calibracion de tu modelo, recuerda que deberas elegir una tasa de interes, por lo que es importante que tu modelo este calibrado, de no estarlo busca como podrias calibrarlo, o justifica por que no lo calibras y como afectaria los resultados

### Modelo no Interpretable

#### Calibracion del modelo

En esta seccion revisa la calibracion de tu modelo, recuerda que deberas elegir una tasa de interes, por lo que es importante que tu modelo este calibrado, de no estarlo busca como podrias calibrarlo, o justifica por que no lo calibras y como afectaria los resultados

## Explicacion de Variables

Realiza una explicacion de que variables son importantes, y como podria utilizarlas el banca para explicar a la comision bancaria y regulatoria por que se otorgo o no el prestamo. Pon atencion a que set usaras para esta parte (train, val, test, calibracion, otro?)